In [67]:
%load_ext autoreload
%autoreload 2

import argparse
import os
import copy
import sys
import time
from datetime import datetime
import torch
import random
import numpy as np

from matplotlib import pyplot as plt

from typing import Tuple
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader

import sys, os
sys.path.append(os.getcwd())
# sys.path.append(os.path.join(os.getcwd(), '../'))
from common.utils import HDF5Dataset, DataCreator
from experiments.models_cnn import CNN, ResNet, BasicBlock1d
from experiments.models_fno import FNO1d
from experiments.train_helper import *
from equations.PDEs import PDE, KdV, KS, Heat
from common.augmentation import Subalgebra
from experiments.train import parse_options as lpsda_parse_options, main as lpsda_main



import os, sys
sys.path.append(os.path.join(os.getcwd(), '../sympde'))
from data.dataset import PDEDataset, PDEDataModule
from data.utils import d_to_LT

from model.setup import setup_model
from model.networks.fno import FNO1d
from model.learner import Learner

from viz.plot_pde_data import plot_1ds

from run import parse_options, main

from data.utils import d_to_coords
from data.pde_data_aug import augment_pde1, augment_KdV
from data.utils import load_obj, save_obj


Set seed!
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### LPSDA

In [68]:
args_lpsda = lpsda_parse_options(notebook = True)
args_lpsda.train_samples=10
args_lpsda.suffix = 'easy'
args_lpsda.device = 'cpu'
args_lpsda.num_workers = 0
args_lpsda.KdV_augmentation = [0, 0.0, 0.4, 0.0]
# args_lpsda.KdV_augmentation = [0, 0.0, 0.0, 0.0]

ret_lpsda = lpsda_main(args_lpsda)

dataloader shapes 1 1 1
Training on dataset data/KdV_train_10_easy.h5
models/FNO1d_KdV_samples10_augmentation0010_shiftfourier_future20_time11271445.pt
Number of parameters: 10856084
Epoch 0
Starting epoch 0...
test
start time [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
pf_steps 0


In [69]:
## CHECK!
data_lpsda = ret_lpsda['train_dataset'][0]

In [70]:
# batch_lpsda = next(iter(ret_lpsda['train_loader']))

In [71]:
# lpsda_data = ret_lpsda['data']
u_lpsda = ret_lpsda['data']

### SymPDE

In [72]:
args = parse_options(notebook=True)
args.pde_name = 'KdV'
args.seed = 42
args.version = f'lpsda_merge'
args.train = True
args.log_dir = '../logs'
args.data_dir = '../../data'
args.batch_size = 10
persistent_workers = False
args.num_workers = 0
args.generators = True

ret = main(args)

Seed set to 42


Augmenting KdV!
Augmenting KdV!
Not augmenting KdV!
Not augmenting KdV!


In [73]:
# (u, dx, dt) = ret['train_dataset'][0]
# us = u.unsqueeze(0)
# batch = (us, dx, dt)

datamodule = ret['datamodule']
batch = next(iter(datamodule.train_dataloader()))
u = batch[0]

In [74]:
# data, labels = ret['model'](batch)

### Compare

In [75]:
print('### LPSDA ###')
batch_lpsda = next(iter(ret_lpsda['train_loader']));
u_batch_lpsda = batch_lpsda[0]

### LPSDA ###


In [76]:
print('### IMPLEMENTATION ###')
batch = next(iter(datamodule.train_dataloader()));
u_batch = batch[0] 

### IMPLEMENTATION ###


In [77]:
# print('### COMPARE ###')

# u_new = torch.load('u_new.pt')
# u_new_lpsda = torch.load('u_new_lpsda.pt')

# a, b = u_new, u_new_lpsda
# # a, b = a.float(), b.float()
# print(a.shape, b.shape)
# print(a.dtype, b.dtype)

# print((a == b).all())
# print(a == b)

# plt.imshow(a - b)
# plt.colorbar()
# plt.show()

# plt.imshow(a)
# plt.show()
# plt.imshow(b)
# plt.show()

In [78]:
# print('### COMPARE ###')

# a, b = u_batch_lpsda, u_batch
# print(a.shape, b.shape)
# print(a.dtype, b.dtype)

# print((a == b).all())
# print(a == b)

# i = -1
# a, b = a[i], b[i]

# plt.imshow((a - b))
# plt.colorbar()
# plt.show()


# plt.imshow(a)
# plt.show()

# plt.imshow(b)
# plt.show()

### Check

In [79]:
# # CHECK1
# for i in range(3):
#     r = ret['train_dataset']
#     r_lpsda = ret_lpsda['train_dataset']
#     print(r[0][i] == r_lpsda[0][i])


In [80]:
print((u == u_lpsda).all())
u == u_lpsda

tensor(True)


tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

## Change data

In [81]:
# import h5py

In [82]:
# for split in ['valid', 'test']:
#     f = h5py.File(f'data/KdV_{split}_easy.h5', 'r')
#     data = f[split]

#     us, dxs, dts = [], [], []

#     for idx in range(10):
#         u = data['pde_140-256'][idx]
#         x = data['x'][idx]
#         t = data['t'][idx]

#         dx = x[1] - x[0]
#         dt = t[1] - t[0]

#         us.append(u)
#         dxs.append(dx)
#         dts.append(dt)

#     us = np.array(us)
#     dxs = np.array(dxs)
#     dts = np.array(dts)

#     save_obj((us, dxs, dts), f'../../data/{split}/KdV')
